In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130620  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,6.214454e+06
2018-02-28,5.072958e+06
2018-03-31,5.378143e+06
2018-04-30,6.309558e+06
2018-05-31,6.752957e+06
2018-06-30,6.241996e+06
2018-07-31,6.711153e+06
2018-08-31,6.800966e+06
2018-09-30,5.467407e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    6.214454e+06
 2018-02-28    5.072958e+06
 2018-03-31    5.378143e+06
 2018-04-30    6.309558e+06
 2018-05-31    6.752957e+06
 2018-06-30    6.241996e+06
 2018-07-31    6.711153e+06
 2018-08-31    6.800966e+06
 2018-09-30    5.467407e+06
 2018-10-31    5.519028e+06
 2018-11-30    5.029404e+06
 2018-12-31    5.432993e+06
 2019-01-31    5.878700e+06
 2019-02-28    5.492207e+06
 2019-03-31    5.044028e+06
 2019-04-30    5.182382e+06
 2019-05-31    5.738934e+06
 2019-06-30    5.221290e+06
 2019-07-31    5.807748e+06
 2019-08-31    6.175861e+06
 2019-09-30    4.601140e+06
 2019-10-31    5.278976e+06
 2019-11-30    4.780452e+06
 2019-12-31    5.143654e+06
 2020-01-31    5.548034e+06
 2020-02-29    3.169837e+06
 2020-03-31    3.179567e+06
 2020-04-30    3.035694e+06
 2020-05-31    3.955942e+06
 2020-06-30    4.252024e+06
 2020-07-31    4.883680e+06
 2020-08-31    5.852362e+06
 2020-09-30    3.692516e+06
 2020-10-31    4.890780e+06
 2020-11-30    5.173213e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.067803
p-value : 0.029030
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1237.456, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1234.890, Time=0.10 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1237.477, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1238.115, Time=0.06 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1240.699, Time=0.12 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1242.685, Time=0.15 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1235.210, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1236.895, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1237.541, Time=0.15 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1238.930, Time=0.13 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=1241.315, Time=0.27 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1236.303, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=1237.170, Time=0.14 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1239.312, Time=0.19 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.58 sec
 ARIMA(3,1,0)(0

ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,1,1))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(0, 1, 1)   Log Likelihood                -614.798
Method:                       css-mle   S.D. of innovations         785450.063
Date:                Sat, 11 Sep 2021   AIC                           1235.595
Time:                        02:43:19   BIC                           1240.736
Sample:                             1   HQIC                          1237.467
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.242e+04    7.5e+04     -0.166      0.868   -1.59e+05    1.35e+05
ma.L1.D.y     -0.3990      0.201     -1.989      0.047      -0.792      -0.006
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([5586203.46574217, 5573782.53642022]), array([785450.06344748, 916368.5904309 ]), array([[4046749.62973042, 7125657.30175392],
       [3777733.10261193, 7369831.97022851]]))
